In [1]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('words')


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tuning_model_class import w2v_tuning
from transformers import  ElectraTokenizerFast
from nltk.corpus import wordnet, stopwords, words
import json
import pandas as pd

[nltk_data] Downloading package words to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


run


In [2]:
vocabulary = words.words()

In [5]:
w2vmodel = w2v_tuning()
w2vmodel.set_vector_path("w2v.42B.300d.txt",'GloVe.wordvectors')
w2vmodel.CreateW2V_Model()

load pre-train model


In [6]:
w2vmodel.model.wv.most_similar('event',topn=5)

[('events', 0.8563973903656006),
 ('venue', 0.6610988974571228),
 ('weekend', 0.6492050290107727),
 ('celebration', 0.6340488195419312),
 ('show', 0.620791494846344)]

In [7]:
# db_schema = ['zipcode','state','company','product','date']
db_schema = ['id','name','age','song','show','order']

In [8]:
target_word = 'How many singer in ABC concert?'
# target_word = 'Who play LOVE in ABC tour?'
target_word = 'Which name of singer that play in event?'

In [3]:
def preprocess_question(text:str):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [WordNetLemmatizer().lemmatize(word, wordnet.NOUN) for word in tokens if (word not in stop_words and word.isalnum())]
    return filtered_tokens

In [4]:
def tokenize_column(col_list:list):
    tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
    token_col = [tokenizer.tokenize(col_name) for col_name in col_list]
    return token_col

In [9]:
for word in preprocess_question(target_word):
    similarities = {}
    for w in db_schema:
        similarities[w] = w2vmodel.model.wv.similarity(word,w)

    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(f"'{word}' similar '{sorted_similarities[0][0]} {sorted_similarities[0][1]:.2f}'")

'name' similar 'name 1.00'
'singer' similar 'song 0.63'
'play' similar 'song 0.52'
'event' similar 'show 0.62'


In [ ]:
# can can redirect to fullcolumn name (with key)

In [10]:
db_id = "concert_singer"
questions = []

with open("spider/dev.json", 'r') as json_file:
    data = json.load(json_file)

for element in data:
    if element['db_id'] == db_id:
        questions.append(element['question'])

In [11]:
import sqlite3

# Specify the full path to the SQLite database file
db_file_path = 'db/concert_singer/concert_singer.sqlite'
db_dict = {}
col_to_tables = {}

# Connect to the database
connection = sqlite3.connect(db_file_path)

# Create a cursor object
cursor = connection.cursor()

# Query the SQLite master table to get the list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
table_names = cursor.fetchall()

# Iterate through the table names
for table in table_names:
    table_name = table[0]
    print(f"Table: {table_name}")

    # Query the PRAGMA statement to get column information for the current table
    cursor.execute(f"PRAGMA table_info({table_name})")

    # Fetch column information
    column_info = cursor.fetchall()

    # Extract and print column names for the current table
    column_names = [col[1] for col in column_info]
    print("Columns:", column_names)
    # print("Tokenize columns:", tokenize_column(column_names))
    db_dict[table_name] = column_names
    
    for col in column_names:
        col_to_tables[col] = table_name

    print("\n")

# Close the cursor and the connection
cursor.close()
connection.close()


Table: stadium
Columns: ['Stadium_ID', 'Location', 'Name', 'Capacity', 'Highest', 'Lowest', 'Average']


Table: singer
Columns: ['Singer_ID', 'Name', 'Country', 'Song_Name', 'Song_release_year', 'Age', 'Is_male']


Table: concert
Columns: ['concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year']


Table: singer_in_concert
Columns: ['concert_ID', 'Singer_ID']




In [12]:
db_dict

{'stadium': ['Stadium_ID',
  'Location',
  'Name',
  'Capacity',
  'Highest',
  'Lowest',
  'Average'],
 'singer': ['Singer_ID',
  'Name',
  'Country',
  'Song_Name',
  'Song_release_year',
  'Age',
  'Is_male'],
 'concert': ['concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year'],
 'singer_in_concert': ['concert_ID', 'Singer_ID']}

In [13]:
col_to_tables

{'Stadium_ID': 'concert',
 'Location': 'stadium',
 'Name': 'singer',
 'Capacity': 'stadium',
 'Highest': 'stadium',
 'Lowest': 'stadium',
 'Average': 'stadium',
 'Singer_ID': 'singer_in_concert',
 'Country': 'singer',
 'Song_Name': 'singer',
 'Song_release_year': 'singer',
 'Age': 'singer',
 'Is_male': 'singer',
 'concert_ID': 'singer_in_concert',
 'concert_Name': 'concert',
 'Theme': 'concert',
 'Year': 'concert'}

In [14]:
questions[:10]

['How many singers do we have?',
 'What is the total number of singers?',
 'Show name, country, age for all singers ordered by age from the oldest to the youngest.',
 'What are the names, countries, and ages for every singer in descending order of age?',
 'What is the average, minimum, and maximum age of all singers from France?',
 'What is the average, minimum, and maximum age for all French singers?',
 'Show the name and the release year of the song by the youngest singer.',
 'What are the names and release years for all the songs of the youngest singer?',
 'What are all distinct countries where singers above age 20 are from?',
 'What are  the different countries with singers above age 20?']

In [44]:
def compare_word_col(df, question:str):
    
    q_tokens = preprocess_question(question)
    for qtoken in q_tokens:
        is_value_exist = df['Q_token'].isin([q_tokens])

        if not is_value_exist.any():    #word is not exist
            list_score = [qtoken]
            tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
            for col_name in list(df.columns):
                if col_name == "Q_token":
                    continue
                score = 0
                for ctoken in tokenizer.tokenize(col_name):
                    similar_score = w2vmodel.model.wv.similarity(ctoken,qtoken)
                    if score < similar_score:
                        score = similar_score
                
                list_score.append(score)
                

            df.loc[len(df.index)] = list_score
            # Reset the index to reindex the DataFrame
            df.reset_index(drop=True, inplace=True)


    return df

In [45]:
cols = ['Q_token'] + list(col_to_tables.keys())
df = pd.DataFrame(columns=cols)

score_df = compare_word_col(df,q)
score_df

,Q_token,Stadium_ID,Location,Name,Capacity,Highest,Lowest,Average,Singer_ID,Country,Song_Name,Song_release_year,Age,Is_male,concert_ID,concert_Name,Theme,Year
0,average,0.259954,0.368489,0.309350,0.445303,0.582857,0.580766,1.000000,0.259954,0.447314,0.309350,0.582450,0.480473,0.471833,0.259954,0.309350,0.213796,0.582450
1,minimum,0.324951,0.347556,0.321239,0.507298,0.542771,0.546494,0.629308,0.324951,0.330326,0.321239,0.484709,0.442823,0.425425,0.324951,0.321239,0.280576,0.484709
2,maximum,0.297724,0.359781,0.318655,0.621605,0.571991,0.514929,0.573869,0.297724,0.298384,0.318655,0.434341,0.418689,0.447453,0.297724,0.318655,0.250972,0.434341
3,age,0.314808,0.421601,0.452403,0.324092,0.386794,0.335882,0.480473,0.350259,0.477531,0.452403,0.565740,1.000000,0.518194,0.314808,0.452403,0.342263,0.565740
4,singer,0.227184,0.229849,0.415831,0.173727,0.209600,0.120408,0.225402,1.000000,0.421359,0.633667,0.633667,0.350259,0.410767,0.545982,0.545982,0.273864,0.375702
5,france,0.269233,0.432113,0.304058,0.209303,0.306740,0.281372,0.266392,0.269233,0.505156,0.304058,0.372878,0.361276,0.360865,0.302260,0.304058,0.215936,0.372878


In [60]:
n = 5
threshold = 0.6

for q in questions[:2]:
    print("Question: ",q)
    ex_match = {}
    pt_match = {}
    q_tokens = preprocess_question(q)
    cols = ['Q_token'] + list(col_to_tables.keys())
    df = pd.DataFrame(columns=cols)
    score_df = compare_word_col(df,q)
    numeric_score_df = score_df.iloc[:, 1:]
    top_columns = numeric_score_df.apply(lambda row: row.nlargest(n).index.tolist(), axis=1)
    top_values = numeric_score_df.apply(lambda row: row.nlargest(n).tolist(), axis=1)
    nlq_token = score_df['Q_token'].tolist()

    for i,w in enumerate(nlq_token):
        for col,val in zip(top_columns[i],top_values[i]):
            if val == 1:
                ex_match[col] = val
            elif val > threshold:
                # print(f"'{w}' similar '{col}' from '{col_to_tables[col]} Table' with [{val:.2f}]")
                pt_match[col] = val
    print("Exact match",ex_match)
    print("Partial match",pt_match)
    print("\n")

Question:  How many singers do we have?
Exact match {'Singer_ID': 1.0}
Partial match {'Is_male': 0.614467203617096, 'Song_Name': 0.6336669921875, 'Song_release_year': 0.6336669921875}


Question:  What is the total number of singers?
Exact match {'Singer_ID': 1.0}
Partial match {'Average': 0.6665894985198975, 'Song_Name': 0.6336669921875, 'Song_release_year': 0.6336669921875}




In [66]:
def map_column_table(nlq,db_dict,score_df,threshold=0.6,topn=5):

    # print("Question: ",nlq)
    ex_match = {}
    pt_match = {}
    numeric_score_df = score_df.iloc[:, 1:]
    top_columns = numeric_score_df.apply(lambda row: row.nlargest(topn).index.tolist(), axis=1)
    top_values = numeric_score_df.apply(lambda row: row.nlargest(topn).tolist(), axis=1)
    nlq_token = score_df['Q_token'].tolist()

    for i,w in enumerate(nlq_token):
        for col,val in zip(top_columns[i],top_values[i]):
            if val == 1:
                ex_match[col] = val
            elif val > threshold:
                pt_match[col] = val
    print("Exact match:",ex_match)
    print("Partial match:",pt_match)
    print("\n")
    match_col = set(list(ex_match.keys()) + list(pt_match.keys()))
    return list(match_col)

In [67]:
cols = ['Q_token'] + list(col_to_tables.keys())
df = pd.DataFrame(columns=cols)

q = questions[0]
print("Question:", q)

q_tokens = preprocess_question(q)
print("Question tokens:", q_tokens)

score_df = compare_word_col(df,q)
match_col = map_column_table(q,db_dict,score_df)

Question: How many singers do we have?
Question tokens: ['many', 'singer']
Exact match: {'Singer_ID': 1.0}
Partial match: {'Is_male': 0.614467203617096, 'Song_Name': 0.6336669921875, 'Song_release_year': 0.6336669921875}


